In [30]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [31]:
import PyPDF2

In [54]:
path='AyushKumar20213086.pdf'

In [55]:
pdfReader=PyPDF2.PdfReader(path)

In [33]:
data=''
for page in pdfReader.pages:
    data=data+'/n/n/n'+page.extract_text()

In [34]:
print(data)

/n/n/nAyush Kumar
Prayagraj, Uttar Pradesh
/ne+91-9693193744 ayush110903kumar@gmail.com Linkedin /gtbGithub
EDUCATION
Motilal Nehru National Institute of Technology, Allahabad 2021 – 2025
B.Tech -Mechanical Engineering- CGPA - 7.81 Prayagraj, Uttar Pradesh
Lord Buddha Public School (12th) 2021
Central Board of Secondary Education - Percentage - 93% Bihar, India
Lord Buddha Public School (10th) 2019
Central Board of Secondary Education - Percentage - 94.8% Bihar, India
COURSEWORK / SKILLS
•Operating Systems
•Object Oriented Programming•Database Management System
•Algorithms and Data Structures
PROJECTS
GoFoodie |React.js, Bootstrap, Node.js, Express.js, MongoDB Nov 2023-Feb 2024
•Developed a responsive web application enabling users to browse food items, manage shopping carts, and
complete purchases seamlessly.
•Implemented user authentication and authorization with JWT, and developed RESTful APIs for handling core
functionalities.
•Designed a visually engaging interface using React.js 

In [35]:
section_keywords = [
    "education", "projects", "skills", "work experience", "experience",
    "certifications", "achievements", "internship", "courses",
    "languages", "summary", "profile", "objective", "extracurricular",
    "publications", "interests"
]

In [36]:
clean_data = data.lower()

In [37]:
import re

section_positions = {}

for keyword in section_keywords:
    pattern = r"\b" + re.escape(keyword) + r"\b"
    match = re.search(pattern, clean_data)
    if match:
        section_positions[keyword] = match.start()
    if not section_positions:
        print("No section headings detected.")


In [38]:
sorted_sections = sorted(section_positions.items(), key=lambda x: x[1])

In [39]:
section_texts = {}
n = len(sorted_sections)

for i in range(n):
    section_name, start_idx = sorted_sections[i]

    # end of this section = start of next
    end_idx = sorted_sections[i + 1][1] if i + 1 < n else len(clean_data)

    section_content = data[start_idx:end_idx].strip()
    section_texts[section_name] = section_content

In [40]:
for section, content in section_texts.items():
    print("\n\n----------------------")
    print(section.upper())
    print("----------------------")
    print(content)



----------------------
EDUCATION
----------------------
EDUCATION
Motilal Nehru National Institute of Technology, Allahabad 2021 – 2025
B.Tech -Mechanical Engineering- CGPA - 7.81 Prayagraj, Uttar Pradesh
Lord Buddha Public School (12th) 2021
Central Board of Secondary Education - Percentage - 93% Bihar, India
Lord Buddha Public School (10th) 2019
Central Board of Secondary Education - Percentage - 94.8% Bihar, India
COURSEWORK /


----------------------
SKILLS
----------------------
SKILLS
•Operating Systems
•Object Oriented Programming•Database Management System
•Algorithms and Data Structures


----------------------
PROJECTS
----------------------
PROJECTS
GoFoodie |React.js, Bootstrap, Node.js, Express.js, MongoDB Nov 2023-Feb 2024
•Developed a responsive web application enabling users to browse food items, manage shopping carts, and
complete purchases seamlessly.
•Implemented user authentication and authorization with JWT, and developed RESTful APIs for handling core
functional

In [41]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [42]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [43]:
vectordb = Chroma(
    collection_name="section_texts",
    persist_directory="my_chroma_db",
    embedding_function=embeddings,
)

In [44]:
docs = []
metas = []
ids = []

for i, (section_name, section_text) in enumerate(section_texts.items()):
    docs.append(section_text)
    metas.append({"section": section_name})
    ids.append(f"sec_{i}")

In [45]:
vectordb.add_texts(
    texts=docs,
    metadatas=metas,
    ids=ids
)

['sec_0', 'sec_1', 'sec_2', 'sec_3', 'sec_4', 'sec_5', 'sec_6']

In [46]:
vectordb.persist()
print("Stored all sections successfully!")

Stored all sections successfully!


In [47]:
query = "Show me the all projects."

results = vectordb.similarity_search(query, k=5)

for r in results:
    print("\n--- Retrieved Section ---")
    print("Section:", r.metadata["section"])
    print(r.page_content)


--- Retrieved Section ---
Section: languages
Languages : C, C++, JavaScript
•Front-end Development : HTML, CSS, React
•Back-end Development : Node.js, Express.js•Database : MongoDB, MySQL
•Version Control : Git, GitHub
•Soft Skills : Creativity, Problem Solving, Clarity of
Thinking
CODING

--- Retrieved Section ---
Section: profile
PROFILE
GeeksforGeeks - Solved 650+ problems.
LeetCode - Solved 480+ problems, contest rating: 1908
POSITIONS OF RESPONSIBILITY AND

--- Retrieved Section ---
Section: education
EDUCATION
Motilal Nehru National Institute of Technology, Allahabad 2021 – 2025
B.Tech -Mechanical Engineering- CGPA - 7.81 Prayagraj, Uttar Pradesh
Lord Buddha Public School (12th) 2021
Central Board of Secondary Education - Percentage - 93% Bihar, India
Lord Buddha Public School (10th) 2019
Central Board of Secondary Education - Percentage - 94.8% Bihar, India
COURSEWORK /

--- Retrieved Section ---
Section: skills
SKILLS
•Operating Systems
•Object Oriented Programming•Database Ma

In [48]:
from langchain_groq import ChatGroq  
from langchain.prompts import PromptTemplate      
from dotenv import load_dotenv    
from langchain_core.output_parsers import StrOutputParser
load_dotenv()  

True

In [49]:
llm=ChatGroq(
    model='llama-3.1-8b-instant'
)

In [53]:
prompt=PromptTemplate(
    template='''I will provide you question and the document which is a resume. You have to analyze it carefully and 
    give me ATS score od it after accurately calculating it. Also answer the question asked, suggest some imporvemnets 
    that can be made, and show some missing skills that the user my have according to job description.
    Question: {question}
    Document: {document}''',
    input_variables=['question', 'document']
)

In [51]:
parser=StrOutputParser()

In [52]:
chain=prompt | llm | parser
answer=chain.invoke({'question':query, 'document':data})
answer

"To calculate the ATS (Applicant Tracking System) score, we need to analyze the resume based on various factors such as keyword usage, formatting, and relevance. Here's a breakdown of the analysis:\n\n**Keyword Analysis:**\n\n* The resume uses relevant keywords from the job description, such as:\n\t+ Programming languages: C, C++, JavaScript, HTML, CSS\n\t+ Frameworks: React.js, Node.js, Express.js\n\t+ Databases: MongoDB, MySQL\n\t+ Version control: Git, GitHub\n\t+ Soft skills: Creativity, Problem Solving, Clarity of Thinking\n* The resume also includes relevant keywords for the industry, such as:\n\t+ MERN stack\n\t+ IoT devices\n\t+ Real-time data acquisition and analysis\n\n**ATS Score Calculation:**\n\n* Keyword usage: 8/10 (the resume uses relevant keywords, but some of them are not used consistently throughout the document)\n* Formatting: 7/10 (the resume has a clear and concise format, but some sections could be better organized)\n* Relevance: 9/10 (the resume is highly releva